# Mini-Project: Geospatial Data Science

We have collected the trajectories from Uber annotations done by 155 students working on the bonus project, and for each screenshot we keep a good trajectory. Those with no good trajectory has been filtered.

We obtain 3903 trajectories in total, which have been put in `trajectories_epsg_3857.txt`, where each row contains `img_id` and corresponding trajectory in EPSG:3857

Example:\
`Participant2_2019_March2019_Shot0146.jpg:-9665838.908 3954722.303,-9667308.043 3953792.864,-9667400.48 3953250.484,-9666948.894 3952362.957,-9666948.894 3952362.957`

### Data Downland and Loading

In [15]:
# get trajectories file from Github repo
import requests

url = 'https://github.com/jalal1/geospatial_data_science_mini/raw/main/trajectories_epsg_3857.txt'
req = requests.get(url, allow_redirects=True)

open('trajectories_epsg_3857.txt', 'wb').write(req.content)

773390

In [16]:
!head -n 5 trajectories_epsg_3857.txt

Participant2_2019_March2019_Shot0146.jpg:-9665838.908 3954722.303,-9667308.043 3953792.864,-9667400.48 3953250.484,-9666948.894 3952362.957,-9666948.894 3952362.957
Participant2_2019_May2019_Shot0148.jpg:-9661613.484 3959618.361,-9661979.179 3960875.051,-9661706.028 3961405.686,-9662401.366 3962442.481,-9662401.366 3962442.481
Participant2_2019_November2019_Shot0134.jpg:-9661378.914 3962191.117,-9661968.194 3962751.86,-9660970.209 3963241.585,-9660354.064 3963819.335,-9660354.064 3963819.335
Participant2_2019_November2019_Shot0203.jpg:-9661610.025 3959722.361,-9661068.311 3959754.904,-9661137.787 3961117.386,-9661301.74 3961428.56,-9661441.175 3962836.023,-9661516.827 3963156.051,-9661441.372 3963138.236,-9661822.419 3962846.053,-9662143.926 3963414.731,-9661685.154 3963684.63,-9661685.154 3963684.63
Participant2_2019_November2019_Shot0275.jpg:-9665575.606 3954158.714,-9667256.791 3953838.871,-9666680.828 3957222.544,-9666172.283 3958726.635,-9666172.283 3958726.635


In [17]:
# read all trajectories from file into dictionary 'img2trajectory'
import os
img2trajectory = {} # {img_id:trajectory} 
n_line = 0

with open('trajectories_epsg_3857.txt', 'r') as f:
    for line in f:
      n_line += 1
      line_values = line.split(":")
      img_id = line_values[0]
      traj = line_values[1]
      img2trajectory[img_id] = []
      for x in traj.split(','):
        lat, lon = x.split(" ")
        img2trajectory[img_id].append((float(lat), float(lon)))

print("Number of trajectories: ", len(img2trajectory))

Number of trajectories:  3903


In [18]:
print(img2trajectory)

{'Participant2_2019_March2019_Shot0146.jpg': [(-9665838.908, 3954722.303), (-9667308.043, 3953792.864), (-9667400.48, 3953250.484), (-9666948.894, 3952362.957), (-9666948.894, 3952362.957)], 'Participant2_2019_May2019_Shot0148.jpg': [(-9661613.484, 3959618.361), (-9661979.179, 3960875.051), (-9661706.028, 3961405.686), (-9662401.366, 3962442.481), (-9662401.366, 3962442.481)], 'Participant2_2019_November2019_Shot0134.jpg': [(-9661378.914, 3962191.117), (-9661968.194, 3962751.86), (-9660970.209, 3963241.585), (-9660354.064, 3963819.335), (-9660354.064, 3963819.335)], 'Participant2_2019_November2019_Shot0203.jpg': [(-9661610.025, 3959722.361), (-9661068.311, 3959754.904), (-9661137.787, 3961117.386), (-9661301.74, 3961428.56), (-9661441.175, 3962836.023), (-9661516.827, 3963156.051), (-9661441.372, 3963138.236), (-9661822.419, 3962846.053), (-9662143.926, 3963414.731), (-9661685.154, 3963684.63), (-9661685.154, 3963684.63)], 'Participant2_2019_November2019_Shot0275.jpg': [(-9665575.606, 

In [19]:
img2trajectory['Participant2_2019_May2019_Shot0122.jpg']



[(-9662001.091, 3958587.93),
 (-9660083.344, 3964436.938),
 (-9660083.344, 3964436.938)]

### Using PyProj package

Cartographic projections and coordinate transformations library, https://pyproj4.github.io/pyproj/stable/#

We may use PyProj to transform one coordinate system to another.

In [20]:
! pip install pyproj

In [25]:
# Example: Convert from EPSG:3857 to EPSG:4326, check here: https://epsg.io/ for more details.
from pyproj import Transformer

def convert_epsg(source_epsg, dist_epsg, x, y):
    transformer = Transformer.from_crs(source_epsg, dist_epsg)
    x, y = transformer.transform(x, y)
    return x, y

input_coords = (-9662852.767, 3962856.202)
convert_epsg('epsg:3857', 'epsg:4326', input_coords[0], input_coords[1])

(33.507460522863106, -86.8028832879271)

we used https://www.georeferencer.com/ to get the coordinates. The format was epsg:3857

We need to convert to GPS coordinates in epsg:4326

In [32]:
from math import e
# read all trajectories from file into dictionary 'img2trajectory'
import os
from pyproj import Transformer
img2trajectory = {} # {img_id:trajectory} 
n_line = 0



with open('trajectories_epsg_3857.txt', 'r') as f:
    for line in f:
      n_line += 1
      line_values = line.split(":")
      img_id = line_values[0]
      traj = line_values[1]
      img2trajectory[img_id] = []
      for x in traj.split(','):
        lat, lon = x.split(" ")
        img2trajectory[img_id].append((float(lat), float(lon)))


new_img2trajectory2 = {}
with open('given_images.txt', 'r') as nf:
  for i in nf:
    i = i.strip()
    new_img2trajectory2[i] = img2trajectory.get(i)

# print(new_img2trajectory2)


def convert_epsg(source_epsg, dist_epsg, x, y):
    transformer = Transformer.from_crs(source_epsg, dist_epsg)
    x, y = transformer.transform(x, y)
    return x, y


img2trajectory_4326 = {}
cnvt_list = []
#print(new_img2trajectory2)

for key, values in new_img2trajectory2.items():
  lst = []
  for value in values:
    x, y = value
    new_lan, new_lon = convert_epsg('epsg:3857', 'epsg:4326', x, y)
    lst.append(((float(new_lan), float(new_lon))))
  img2trajectory_4326[key] = lst

print(img2trajectory_4326)
    
 


{'Participant2_2019_March2019_Shot0120.jpg': [(89.88360380757406, -86.79085880784314), (33.5190296898996, -86.79463909822177), (33.4824028301108, -86.7832609829342), (33.493768499871415, -86.78502755587301), (33.51567060922143, 86.8028978945336)], 'Participant2_2019_May2019_Shot0122.jpg': [(33.47548407394474, -86.79523255224791), (33.51929986513208, -86.77800513783616), (33.51929986513208, -86.77800513783616)], 'Participant2_2019_May2019_Shot0193.jpg': [(0.0003011116549400449, -0.0007795354064379487), (0.00030118731374821834, -0.0007793822364954839), (0.000301192020920307, -0.0007792762164273369), (0.000301192020920307, -0.0007792762164273369)], 'Participant2_2019_November2019_Shot0178.jpg': [(33.511521401709146, -86.78798410011927), (89.99494322363327, -86.82500256805002), (33.50809589835323, -86.78740757933622), (33.50809589835323, -86.78740757933622)], 'Participant2_2019_November2019_Shot0247.jpg': [(33.52563088334829, -86.76850847204595), (33.51975862561191, -86.78190716790209), (3

### Visualizing a trajectory using Folium

Folium allows us to overlay plots on top of a map.

In [33]:
! pip install folium

In [34]:
# plot a trajectory
import folium
import numpy as np

img_id = "Participant2_2019_May2019_Shot0193.jpg" 
traj = img2trajectory_4326[img_id]

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 5
# if trajectory is not complete, reduce start_level until you see the entire trajectory
# if trajectory is too small, increase start_level

map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

map

In [35]:

import time
from IPython.display import clear_output

pos = 0
while pos < len(traj):
    folium.Marker(traj[pos]).add_to(map)
    pos += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.5)

In [36]:
# Below is an example of one image with an issue in the trajectory. We need to find such cases
img_id = 'Participant2_2019_October2019_Shot0270.jpg'

# In case you want to check more images. Sample a random image and check the trajectory
# import random
# img_id = random.choice(list(img2trajectory.keys()))

# convert coordinates
traj = []
for point in img2trajectory[img_id]:
  lat, lon = convert_epsg('epsg:3857', 'epsg:4326', point[0], point[1])
  traj.append((lat, lon))

center = np.asarray(traj[0]) + np.asarray(traj[-1])
center /= 2

start_level = 13
# if trajectory is not complete, reduce start_level until you see the entire trajectory
# if trajectory is too small, increase start_level

map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

pos = 0
while pos < len(traj):
    folium.Marker(traj[pos]).add_to(map)
    pos += 1
    display(map)
    clear_output(wait=True)
    time.sleep(0.5)

In [37]:
# plot a trajectory
import folium
import numpy as np
import time
from IPython.display import clear_output

for key in img2trajectory_4326:
  print(key)
  time.sleep(0.1)
  img_id = key
  traj = img2trajectory_4326[img_id]

  center = np.asarray(traj[0]) + np.asarray(traj[-1])
  center /= 2

  start_level = 13
# if trajectory is not complete, reduce start_level until you see the entire trajectory
# if trajectory is too small, increase start_level

  map = folium.Map(zoom_start=start_level, location=center)
  folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)

  pos = 0
  while pos < len(traj):
    folium.Marker(traj[pos]).add_to(map)
    pos += 1
    display(map)
    clear_output(wait=True)
    #print(key)
  time.sleep(0.2)



### Fast map matching (FMM)*

Reference: Can Yang & Gyozo Gidofalvi (2018) Fast map matching, an algorithm
integrating hidden Markov model with precomputation, International Journal of Geographical Information Science, 32:3, 547-570, DOI: 10.1080/13658816.2017.1400548

https://fmm-wiki.github.io/

As you may have noticed, the trajectories are not aligned with the underlying road network. i.e., a line between two consecutive points is a straight line and does not follow the actual roads.

To do match those points to the road network, and add back intermediate points, we use FMM. First, we need to install it.

**Install Prerequisites**

In [ ]:
### TODO ###

# install FMM "requirements" following https://fmm-wiki.github.io/docs/installation
# You need to choose the right OS on that page
# For example, on Ubuntu you can run
# !sudo apt-get install libboost-dev libboost-serialization-dev gdal-bin libgdal-dev make cmake libbz2-dev libexpat1-dev swig python-dev

In [38]:
# Install all the requirements with:
! sudo apt-get install libboost-dev libboost-serialization-dev \
gdal-bin libgdal-dev make cmake libbz2-dev libexpat1-dev swig python-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libboost-dev is already the newest version (1.65.1.0ubuntu1).
libboost-dev set to manually installed.
make is already the newest version (4.1-9.1ubuntu1).
make set to manually installed.
python-dev is already the newest version (2.7.15~rc1-1).
gdal-bin is already the newest version (2.2.3+dfsg-2).
libboost-serialization-dev is already the newest version (1.65.1.0ubuntu1).
libboost-serialization-dev set to manually installed.
libgdal-dev is already the newest version (2.2.3+dfsg-2).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.2).
libbz2-dev is already the newest version (1.0.6-8.1ubuntu0.2).
libbz2-dev set to manually installed.
libexpat1-dev is already the newest version (2.2.5-3ubuntu0.7).
libexpat1-dev set to manually installed.
The following additional packages will be installed:
  swig3.0
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The followi

**Install FMM**

In [39]:
!git clone https://github.com/cyang-kth/fmm.git

Cloning into 'fmm'...
remote: Enumerating objects: 5162, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 5162 (delta 14), reused 35 (delta 10), pack-reused 5119
Receiving objects: 100% (5162/5162), 15.33 MiB | 28.54 MiB/s, done.
Resolving deltas: 100% (3062/3062), done.


In [40]:
# !!!! This could take ~ 6 mintues !!!!
# it may not work and requires for password, etc.
# in this case, go to your folder using console to compile and install FMM

import os
# change working directory
os.chdir("fmm")

if not os.path.exists('build'):
  os.mkdir('build')
# ! mkdir build
os.chdir("build")
# ! cd build
! cmake ..
! make -j4
! sudo make install

-- CMAKE version 3.12.0
-- Set CMP0074 state to NEW
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- 
No conda environment found in PATH!
PATH=/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin

-- Could NOT find Conda (missing: CONDA_PREFIX) 
-- Non conda exist, search library in default path
-- Found GDAL: /usr/lib/libgdal.so (Required is at l

In [42]:
# Verfication of installation
!fmm

[info][fmm_app_config.cpp:49 ] Start reading FMM configuration from arguments
fmm argument lists:
--ubodt (required) <string>: Ubodt file name
--network (required) <string>: Network file name
--network_id (optional) <string>: Network id name (id)
--source (optional) <string>: Network source name (source)
--target (optional) <string>: Network target name (target)
--gps (required) <string>: GPS file name
--gps_id (optional) <string>: GPS id name (id)
--gps_x (optional) <string>: GPS x name (x)
--gps_y (optional) <string>: GPS y name (y)
--gps_timestamp (optional) <string>: GPS timestamp name (timestamp)
--gps_geom (optional) <string>: GPS geometry name (geom)
--gps_point (optional): if specified read input data as gps point, otherwise (default) read input data as trajectory
--output (required) <string>: Output file name
--output_fields (optional) <string>: Output fields
  opath,cpath,tpath,mgeom,pgeom,
  offset,error,spdist,tp,ep,length,duration,speed,all
-k/--candidates (optional) <int>

In [43]:
# Change to the parent folder which contains fmm_test.py
if os.getcwd() != "/content/fmm/example/python":
  os.chdir("/content/fmm/example/python")
os.system('python fmm_test.py')

256

In [44]:
# Install osmnx package to prepare the network, that will be used as input to FMM
!pip install osmnx

     |████████████████████████████████| 95 kB 2.5 MB/s 
     |████████████████████████████████| 1.0 MB 36.6 MB/s 
     |████████████████████████████████| 11.2 MB 41.9 MB/s 
     |████████████████████████████████| 63 kB 1.2 MB/s 
     |████████████████████████████████| 1.0 MB 49.7 MB/s 
     |████████████████████████████████| 16.7 MB 422 kB/s 
     |████████████████████████████████| 930 kB 66.0 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompa

## Steps to run FMM after installation: 
1- Prepare the network using osmnx \
2- Run the match(...) function

In [45]:
# !!!! This could take ~ 2 mintues !!!!

import osmnx as ox
import time
from shapely.geometry import Polygon
import os
import numpy as np
from fmm import Network,NetworkGraph,STMATCH,STMATCHConfig

def save_graph_shapefile_directional(G, filepath=None, encoding="utf-8"):
    # default filepath if none was provided
    if filepath is None:
        filepath = os.path.join(ox.settings.data_folder, "graph_shapefile")

    # if save folder does not already exist, create it (shapefiles
    # get saved as set of files)
    if not filepath == "" and not os.path.exists(filepath):
        os.makedirs(filepath)
    filepath_nodes = os.path.join(filepath, "nodes.shp")
    filepath_edges = os.path.join(filepath, "edges.shp")

    # convert undirected graph to gdfs and stringify non-numeric columns
    gdf_nodes, gdf_edges = ox.utils_graph.graph_to_gdfs(G)
    gdf_nodes = ox.io._stringify_nonnumeric_cols(gdf_nodes)
    gdf_edges = ox.io._stringify_nonnumeric_cols(gdf_edges)
    # We need an unique ID for each edge
    gdf_edges["fid"] = np.arange(0, gdf_edges.shape[0], dtype='int')
    # save the nodes and edges as separate ESRI shapefiles
    gdf_nodes.to_file(filepath_nodes, encoding=encoding)
    gdf_edges.to_file(filepath_edges, encoding=encoding)

print("osmnx version",ox.__version__)

# !!! Download by a bounding box # !!!
# -------------------------------------------------
bounds = (-86.9671,-86.5901,33.3472,33.6598)
# -------------------------------------------------
x1,x2,y1,y2 = bounds
boundary_polygon = Polygon([(x1,y1),(x2,y1),(x2,y2),(x1,y2)])
G = ox.graph_from_polygon(boundary_polygon, network_type='drive')
start_time = time.time()
save_graph_shapefile_directional(G, filepath='./network-new')
print("--- %s seconds ---" % (time.time() - start_time))

osmnx version 1.1.2


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.


--- 32.09665250778198 seconds ---


In [46]:
# copy network folder to /content/fmm/example/python/network-new
!mv '/content/network-new' '/content/fmm/example/python/network-new'

mv: cannot stat '/content/network-new': No such file or directory


In [47]:
# match(...) function

def match(path,points):
  network = Network(path,"fid","u","v")
  graph = NetworkGraph(network)
  print (graph.get_num_vertices())
  model = STMATCH(network,graph)
  wkt  = str(points)
  config = STMATCHConfig()
  config.k = 4
  config.gps_error = 0.5
  config.radius = 0.4
  config.vmax = 30;
  config.factor =1.5
  result = model.match_wkt(wkt,config)
  print (type(result))
  # print ("Opath ",list(result.opath))
  # print ("Cpath ",list(result.cpath))
  # print ("WKT ",result.mgeom.export_wkt())
  return result.mgeom.export_wkt()

In [48]:
def match_trajs(network_path, traj):
  """
  match [traj] on [network_path]. [traj] should be [(lon1, lat1), (lon2, lat2), ...]
  Output: matched traj as "LINESTRING(-86.797211 33.499194,-86.7973..."
  """
  from shapely.geometry import LineString
  # import geopandas !!

  traj_ = LineString(traj)
  return match(network_path, traj_)

In [49]:
# get a random trajectory
import random
img_id = random.choice(list(img2trajectory_4326.keys()))
traj = img2trajectory_4326[img_id]
print("Original Trajectory: ", traj)
# swap (lat, lon) --> (lon, lat) as FMM requires lon before lat
traj_m = [(sub[1], sub[0]) for sub in traj]
fmm_traj = match_trajs('/content/fmm/example/python/network-new/edges.shp', traj_m)
# remove LINESTRING and extra parentheses
fmm_traj = fmm_traj.split("LINESTRING(")[1]
fmm_traj = fmm_traj.split(")")[0]
print("Matched Trajectory: ", fmm_traj)

Original Trajectory:  [(33.59750233899618, -86.64952343262821), (33.54433413274082, -86.75849721532839), (33.561088694353714, -86.75227761267782)]
32332
<class 'fmm.PyMatchResult'>
Matched Trajectory:  -86.648028 33.597502,-86.648028 33.597414,-86.64805 33.596321,-86.648058 33.595949,-86.648066 33.595906,-86.648074 33.595867,-86.648056 33.595538,-86.648037 33.59542,-86.647976 33.595189,-86.647926 33.595092,-86.647886 33.595014,-86.64774 33.594815,-86.647671 33.594745,-86.647651 33.594618,-86.647664 33.594506,-86.647811 33.594363,-86.648465 33.593977,-86.650263 33.592912,-86.650558 33.592716,-86.650585 33.592698,-86.650778 33.592549,-86.650899 33.592451,-86.65106 33.592328,-86.651267 33.59212,-86.651604 33.591714,-86.651819 33.59144,-86.652233 33.590821,-86.652489 33.590486,-86.652754 33.590202,-86.653127 33.589915,-86.653234 33.589851,-86.653521 33.58967,-86.653976 33.589404,-86.654542 33.589073,-86.654709 33.588975,-86.655152 33.588728,-86.655608 33.588485,-86.655753 33.588408,-86.655

In [50]:
# plot both trajctories, the original and the matched one.

matched_traj = []
for c in fmm_traj.split(","):
    lon, lat = c.split(" ")
    matched_traj.append((float(lat), float(lon)))

center = np.asarray(matched_traj[0]) + np.asarray(matched_traj[-1])
center /= 2

start_level = 13
# if trajectory is not complete, reduce start_level until you see the entire trajectory
# if trajectory is too small, increase start_level

map = folium.Map(zoom_start=start_level, location=center)
folium.PolyLine(traj, color='red', weight=5, opacity=0.8).add_to(map)
folium.PolyLine(matched_traj, color='green', weight=5, opacity=0.8).add_to(map)

map

We can also visualize the trajectories using Folium HeatMap

In [51]:
from folium.plugins import HeatMap

map = folium.Map(zoom_start=10, height=700, location=[33.5186, -86.8104])
HeatMap(matched_traj, blur=5, radius=5).add_to(map)
map

In [87]:
# get a random trajectory
from folium.plugins import HeatMap
matched_traj = []

for key in img2trajectory_4326:
  img_id = key
  traj = img2trajectory_4326[img_id]
  #print("Original Trajectory: ", traj)
  # swap (lat, lon) --> (lon, lat) as FMM requires lon before lat
  traj_m = [(sub[1], sub[0]) for sub in traj]
  fmm_traj = match_trajs('/content/fmm/example/python/network-new/edges.shp', traj_m)
  # remove LINESTRING and extra parentheses
  fmm_traj = fmm_traj.split("LINESTRING(")[1]
  fmm_traj = fmm_traj.split(")")[0]
  #print("Matched Trajectory: ", fmm_traj)
  #print(len(traj))


  # plot both trajctories, the original and the matched one.

  for c in fmm_traj.split(","):
    if(len(c.split(" ")) == 2):
      i = 0
      lon, lat = c.split(" ")
      matched_traj.append((float(lat), float(lon)))

      if(i == 0):
        center = np.asarray(matched_traj[0]) + np.asarray(matched_traj[-1])
        center /= 2

        start_level = 13


#from folium.plugins import HeatMap

map = folium.Map(zoom_start=10, height=700, location=[33.5186, -86.8104])
HeatMap(matched_traj, blur=5, radius=5).add_to(map)
map

32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class 'fmm.PyMatchResult'>
32332
<class '

In [ ]:
# The End